
requirment:
python==3.6.5
flask

In [1]:
import pandas as pd
import numpy as np
import threading
import json
from flask import Flask, jsonify, render_template,request,Markup

class WideTable:
    def __init__(self,df):
        self.df = df
        self.bin_dict = {}

    def var_stat(self,var,splits=None):
        if splits is None: 
            if var not in self.bin_dict:
                categories, bins = pd.qcut(self.df[var], 20, retbins=True,duplicates="drop")
            else:
                bins = self.bin_dict[var]
                categories = pd.cut(self.df[var], bins=bins)
        else:
            bins = [self.df[var].min()-1] + sorted(splits) + [self.df[var].max()+1]
            categories = pd.cut(self.df[var], bins=bins)

        stats = self.df.groupby(categories.cat.codes).agg({"target":[
            "count",                # number of observations
            "mean",                 # bad_rate
            lambda x:(x==1).sum(),  # number of bads
            lambda x:(x==0).sum(),  # number of goods
        ]})
        stats = stats.reset_index()
        stats.columns = ["bin","nobs","bad_rate","bad","good"]
        total_bad,total_good = stats["bad"].sum(), stats["good"].sum()
        stats["woe"] = stats.apply(lambda x:np.log((x["bad"]+1)/total_bad)-np.log((x["good"]+1)/total_good),axis=1)
        stats["iv"] = stats["woe"]*(stats["bad"]/total_bad-stats["good"]/total_good)
        
        stats["interval"] = stats["bin"].map(lambda x:str(categories.cat.categories[x]) if x>=0 else "N/A") 
        info = {
            "var":var,
            "splits":list(categories.cat.categories.left)[1:], # left edge not needed, only the splits
            "max":self.df[var].max(),
            "min":self.df[var].min(),
            "iv":stats["iv"].sum(),
            "total":int(stats["nobs"].sum()),    
        }
                
        return stats, info
    
    def update_bin(self,var,bins):
        self.bin_dict[var] = bins



In [14]:
data1 = pd.read_pickle("./data_sample.pkl")
data1.shape
data1.head()
wt = WideTable(data1)

(20000, 50)

In [ ]:


app = Flask("data_server",static_folder="./flask_file",template_folder="./flask_file",static_url_path="/static")

DATA = {} #{"data1":data1}
html_str = ""

@app.route("/data_list",methods=["GET"])
def get_data():
    return jsonify(list(DATA.keys()))

@app.route("/data_desc/<data_name>",methods=["GET"])
def get_data_desc(data_name):
    desc = {
        "shape":str(DATA[data_name].shape),
        "columns":str(DATA[data_name].columns),
        "indexs":str(DATA[data_name].index),
    }
    return jsonify(desc)

@app.route("/data1/<ind>/<col>",methods=["GET"])
def get_data_by_key(ind,col):
    return jsonify(str(DATA['data1'].loc[ind,col]))

@app.route("/data_set/<key>/<value>",methods=["GET"])
def set_data_by_key(key,value):
    DATA[key] = value
    return jsonify("ok")

@app.route("/test",methods=["GET","POST"])
def test():
    if request.method == "GET":
        var = request.args.get("var")
        stats, info = wt.var_stat(var)
        return render_template(
            "test.html",
            data=Markup({"df":stats.to_dict(orient="records"),"info":info})
        )
    else:
        var = request.args.get("var","")
        splits = json.loads(request.data)
        stats, info = wt.var_stat(var,splits)
        return jsonify({"df":stats.to_dict(orient="records"),"info":info})


app.config["ENV"] = "development"
app.config["TEMPLATES_AUTO_RELOAD"] = True
t=threading.Thread(target=app.run,kwargs={'port':50000})
t.start()
print('running')
# app.run(port=50000,threaded=True)

running * Serving Flask app "data_server" (lazy loading)

 * Environment: development
 * Debug mode: off


 * Running on http://127.0.0.1:50000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Oct/2023 13:12:56] "GET /test?var=bairong_als_699&step=0.1&xtype=log HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:12:58] "POST /test?var=bairong_als_699 HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:13:06] "POST /test?var=bairong_als_699 HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:13:08] "POST /test?var=bairong_als_699 HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:13:09] "POST /test?var=bairong_als_699 HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:13:10] "POST /test?var=bairong_als_699 HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:13:15] "GET /test?var=bairong_als_699&step=1&xtype=log HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:13:18] "POST /test?var=bairong_als_699 HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:30:36] "GET /test?var=bairong_als_699&step=1&xtype=log HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:30:36] "GET /static/range-slider.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2023 13:30:36] "GET

In [7]:
%%HTML
<iframe src="http://127.0.0.1:50000/test?var=var1&step=1&xtype=log" width="100%" height="500px" style="background-color:antiquewhite"></iframe>

127.0.0.1 - - [13/Oct/2023 13:38:30] "GET /test?var=bairong_als_699&step=1&xtype=log HTTP/1.1" 200 -
[2023-10-13 13:38:48,896] ERROR in app: Exception on /test [POST]
Traceback (most recent call last):
  File "d:\Anaconda\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "d:\Anaconda\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "d:\Anaconda\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "d:\Anaconda\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "d:\Anaconda\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "d:\Anaconda\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-b62aef6f1250>", line 42, in test
    